### 

### 11.
The midpoint between positions  P
  and  Q
  is given by the equation  M=0.5∗(P+Q)
 . But this expression appears to violate the assumptions above, stating that addition and scalar multiplication with positions are not meaningful operations. How can one make sense of this equation in terms of purely meaningful operations?

M = P + 0.5*(Q-P)

### 12.
Implement a 2D coordinate management system, in your programming language of choice, where each position must be annotated with a frame. Your system should define an API to create positions and frames, and to retrieve coordinates of any position in any given frame. Frames should be identified with string names. The manager should maintain a list of named coordinate frames, each of which is represented in its coordinates relative to the world coordinate system (call this special frame "world"). Define a Point data structure that stores two elements

coords: the point's coordinates, expressed in terms of its reference frame.

frame: the name of its reference frame.

Do the same for a Directional data structure. Now, implement methods Point.to(frame) and Directional.to(frame) that compute a new Point and Directional, respectively, whose reference frames are now the given frame, and whose coordinates are expressed correctly relative to the new frame.

Extend the coordinate management system of exercise 12 to implement geometric operations, enforcing that each of the items needs to be of the same reference frame. Ensure that Point - Point = Directional, Point + Directional = Directional, Directional +/- Directional = Directional, and Directional * scalar = Directional.

This tutorial for operator overriding in Python will come in handy here.

Extend the coordinate management system of exercise 12 to also include units that should be specified to the Point, Directional, and Frame structure upon construction. Implement support for 'm', 'mm', 'cm' and 'km' arguments indicating meters, millimeters, centimeters, and kilometers.

In [1]:
#Skeleton code for exercise 12

class Frame:
    #TODO: what here?
    pass

#a dictionary from strings to Frame objects, to be used in Point.to and Direction.to
named_frames = dict()

class Point:
    def __init__(self,coords,frame):
        self.coords = coords
        self.frame = frame
    def to(self,newframe):
        """Returns a Point expressing the same physical point in space, but represented in the new frame"""
        #TODO: what here?
        return Point(self.coords,newframe)

class Directional:
    def __init__(self,coords,frame):
        self.coords = coords
        self.frame = frame
    def to(self,newframe):
        """Returns a Directional expressing the same physical direction in space, but represented in the new frame"""
        #what here?
        return Point(self.coords,newframe)

In [2]:
import numpy as np

# 단위 변환 테이블 (모두 미터 기준)
unit_scale = {
    'm': 1.0,
    'cm': 0.01,
    'mm': 0.001,
    'km': 1000.0
}

# 좌표계 정의
class Frame:
    def __init__(self, name, origin_coords=(0, 0), unit='m'):
        self.name = name
        self.origin = np.array(origin_coords) * unit_scale[unit]
        named_frames[name] = self

    def get_origin_in_world(self):
        return self.origin

# 전역 좌표계 프레임 딕셔너리
named_frames = {}

# 위치 클래스
class Point:
    def __init__(self, coords, frame, unit='m'):
        self.coords = np.array(coords) * unit_scale[unit]
        self.frame = frame

    def to(self, newframe):
        if self.frame == newframe:
            return Point(self.coords, self.frame, 'm')
        origin_self = named_frames[self.frame].get_origin_in_world()
        world_coords = origin_self + self.coords
        origin_new = named_frames[newframe].get_origin_in_world()
        new_coords = world_coords - origin_new
        return Point(new_coords, newframe, 'm')

    def __sub__(self, other):
        if not isinstance(other, Point):
            raise TypeError("Subtraction only supported between Points.")
        if self.frame != other.frame:
            raise ValueError("Points must be in the same frame for subtraction.")
        return Directional(self.coords - other.coords, self.frame, 'm')

    def __add__(self, direction):
        if not isinstance(direction, Directional):
            raise TypeError("Can only add Directional to Point.")
        if self.frame != direction.frame:
            raise ValueError("Point and Directional must be in the same frame.")
        return Point(self.coords + direction.coords, self.frame, 'm')

# 방향 클래스
class Directional:
    def __init__(self, coords, frame, unit='m'):
        self.coords = np.array(coords) * unit_scale[unit]
        self.frame = frame

    def to(self, newframe):
        if self.frame != newframe:
            raise ValueError("Directions must remain in the same frame (no origin shift).")
        return Directional(self.coords, newframe, 'm')

    def __add__(self, other):
        if self.frame != other.frame:
            raise ValueError("Frames must match for addition.")
        return Directional(self.coords + other.coords, self.frame, 'm')

    def __sub__(self, other):
        if self.frame != other.frame:
            raise ValueError("Frames must match for subtraction.")
        return Directional(self.coords - other.coords, self.frame, 'm')

    def __mul__(self, scalar):
        return Directional(self.coords * scalar, self.frame, 'm')